# CI/CD for a Kubeflow pipeline on Vertex AI

**Learning Objectives:**
1. Learn how to create a custom Cloud Build builder to pilote Vertex AI Pipelines
1. Learn how to write a Cloud Build config file to build and push all the artifacts for a KFP
1. Learn how to setup a Cloud Build GitHub trigger a new run of the Kubeflow PIpeline

In this lab you will walk through authoring of a **Cloud Build** CI/CD workflow that automatically builds, deploys, and runs a Kubeflow pipeline on Vertex AI. You will also integrate your workflow with **GitHub** by setting up a trigger that starts the  workflow when a new tag is applied to the **GitHub** repo hosting the pipeline's code.

## Configuring environment settings

In [1]:
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]
REGION = "us-central1"
ARTIFACT_STORE = f"gs://{PROJECT_ID}-kfp-artifact-store"

Let us make sure that the artifact store exists:

In [2]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

gs://qwiklabs-gcp-04-853e5675f5e8-kfp-artifact-store/


## Creating the KFP CLI builder for Vertex AI

### Exercise

In the cell below, write a docker file that
* Uses `gcr.io/deeplearning-platform-release/base-cpu` as base image
* Install the python packages `kfp` with version `1.6.6 ` and `google-cloud-aiplatform` with version `1.3.0`
* Starts `/bin/bash` as entrypoint

In [3]:
%%writefile kfp-cli/Dockerfile

# TODO
FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install kfp==1.6.6 google-cloud-aiplatform==1.3.0
ENTRYPOINT ["/bin/bash"]

Overwriting kfp-cli/Dockerfile


### Build the image and push it to your project's **Container Registry**.

In [4]:
KFP_CLI_IMAGE_NAME = "kfp-cli-vertex"
KFP_CLI_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{KFP_CLI_IMAGE_NAME}:latest"
KFP_CLI_IMAGE_URI

'gcr.io/qwiklabs-gcp-04-853e5675f5e8/kfp-cli-vertex:latest'

### Exercise

In the cell below, use `gcloud builds` to build the `kfp-cli-vertex` Docker image and push it to the project gcr.io registry.

In [11]:
!{KFP_CLI_IMAGE_URI}

/bin/bash: gcr.io/qwiklabs-gcp-04-853e5675f5e8/kfp-cli-vertex:latest: No such file or directory


In [12]:
# COMPLETE THE COMMAND
# https://cloud.google.com/sdk/gcloud/reference/builds/submit
!gcloud builds submit --async --timeout 15m --tag {KFP_CLI_IMAGE_URI} kfp-cli

Creating temporary tarball archive of 1 file(s) totalling 142 bytes before compression.
Uploading tarball of [kfp-cli] to [gs://qwiklabs-gcp-04-853e5675f5e8_cloudbuild/source/1646386617.46244-ac76cb5a80954e03b0d5dcd5dd1b78d4.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-gcp-04-853e5675f5e8/locations/global/builds/64c36658-34b1-4956-9c5a-2a761e728236].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/64c36658-34b1-4956-9c5a-2a761e728236?project=1076138843678].
ID                                    CREATE_TIME                DURATION  SOURCE                                                                                                     IMAGES  STATUS
64c36658-34b1-4956-9c5a-2a761e728236  2022-03-04T09:36:58+00:00  -         gs://qwiklabs-gcp-04-853e5675f5e8_cloudbuild/source/1646386617.46244-ac76cb5a80954e03b0d5dcd5dd1b78d4.tgz  -       QUEUED


## Understanding the **Cloud Build** workflow.

### Exercise

In the cell below, you'll complete the `cloudbuild_vertex.yaml` file describing the CI/CD workflow and prescribing how environment specific settings are abstracted using **Cloud Build** variables.

The CI/CD workflow automates the steps you walked through manually during `lab-02_vertex`:
1. Builds the trainer image
1. Compiles the pipeline
1. Uploads and run the pipeline to the Vertex AI Pipeline environment
1. Pushes the trainer to your project's **Container Registry**
 

The **Cloud Build** workflow configuration uses both standard and custom [Cloud Build builders](https://cloud.google.com/cloud-build/docs/cloud-builders). The custom builder encapsulates **KFP CLI**. 

In [20]:
%%writefile cloudbuild_vertex.yaml
# Copyright 2021 Google LLC

# Licensed under the Apache License, Version 2.0 (the "License"); you may not use this
# file except in compliance with the License. You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0
    
# Unless required by applicable law or agreed to in writing, software 
# distributed under the License is distributed on an "AS IS"
# BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either 
# express or implied. See the License for the specific language governing 
# permissions and limitations under the License.

steps:
# Build the trainer image
# TODO
- name: 
  id: 'Build the trainer image'
  args: ['build', '-t', 'gcr.io/$PROJECT_ID/trainer_image_covertype_vertex:latest', '.']
  dir: $_PIPELINE_FOLDER/trainer_image_vertex

# Push the trainer image, to make it available in the compile step
- name: 'gcr.io/cloud-builders/docker'
  id: 'Push the trainer image'
  args: ['push', 'gcr.io/$PROJECT_ID/trainer_image_covertype_vertex:latest']
  dir: $_PIPELINE_FOLDER/trainer_image_vertex

# Compile the pipeline
- name: 'gcr.io/$PROJECT_ID/kfp-cli-vertex'
  id: 'Compile the pipeline'
  args:
  - '-c'
  - |
    dsl-compile-v2 # TODO
  env:
  - 'PIPELINE_ROOT=gs://$PROJECT_ID-kfp-artifact-store/pipeline'
  - 'PROJECT_ID=$PROJECT_ID'
  - 'REGION=$_REGION'
  - 'SERVING_CONTAINER_IMAGE_URI=us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-20:latest'
  - 'TRAINING_CONTAINER_IMAGE_URI=gcr.io/$PROJECT_ID/trainer_image_covertype_vertex:latest'
  - 'TRAINING_FILE_PATH=gs://$PROJECT_ID-kfp-artifact-store/data/training/dataset.csv'
  - 'VALIDATION_FILE_PATH=gs://$PROJECT_ID-kfp-artifact-store/data/validation/dataset.csv'
  dir: pipeline_vertex
  
# Run the pipeline
- name: 'gcr.io/$PROJECT_ID/kfp-cli-vertex'
  args:
  - '-c'
  - |
    python kfp-cli_vertex/run_pipeline.py  # TODO
    
# Push the images to Container Registry
# TODO: List the images to be pushed to the project Docker registry
# TODO
images: ['gcr.io/$PROJECT_ID/trainer_image_covertype_vertex:latest']

# This is required since the pipeline run overflows the default timeout
timeout: 10800s


Overwriting cloudbuild_vertex.yaml


## Manually triggering CI/CD runs

You can manually trigger **Cloud Build** runs using the [gcloud builds submit command]( https://cloud.google.com/sdk/gcloud/reference/builds/submit).

In [21]:
SUBSTITUTIONS = f"_REGION={REGION},_PIPELINE_FOLDER=./"
SUBSTITUTIONS

'_REGION=us-central1,_PIPELINE_FOLDER=./'

In [23]:
!gcloud builds submit . --config cloudbuild_vertex.yaml --substitutions {SUBSTITUTIONS} --async

Creating temporary tarball archive of 19 file(s) totalling 74.4 KiB before compression.
Uploading tarball of [.] to [gs://qwiklabs-gcp-04-853e5675f5e8_cloudbuild/source/1646387873.746347-cd4b828943c44063a86b35bab0c9eac1.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-gcp-04-853e5675f5e8/locations/global/builds/d3aba731-9e62-48ab-921b-474ee184f3cd].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/d3aba731-9e62-48ab-921b-474ee184f3cd?project=1076138843678].
ID                                    CREATE_TIME                DURATION  SOURCE                                                                                                      IMAGES  STATUS
d3aba731-9e62-48ab-921b-474ee184f3cd  2022-03-04T09:57:54+00:00  -         gs://qwiklabs-gcp-04-853e5675f5e8_cloudbuild/source/1646387873.746347-cd4b828943c44063a86b35bab0c9eac1.tgz  -       QUEUED


**Note:** If you experience issues with CloudBuild being able to access Vertex AI, you may need to run the following commands in **CloudShell**:

```
PROJECT_ID=$(gcloud config get-value project)
PROJECT_NUMBER=$(gcloud projects list --filter="name=$PROJECT_ID" --format="value(PROJECT_NUMBER)")
gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member="serviceAccount:$PROJECT_NUMBER@cloudbuild.gserviceaccount.com" \
    --role="roles/aiplatform.user"
gcloud iam service-accounts add-iam-policy-binding \
    $PROJECT_NUMBER-compute@developer.gserviceaccount.com \
    --member="serviceAccount:$PROJECT_NUMBER@cloudbuild.gserviceaccount.com" \
    --role="roles/iam.serviceAccountUser"
```

## Setting up GitHub integration

## Exercise

In this exercise you integrate your CI/CD workflow with **GitHub**, using [Cloud Build GitHub App](https://github.com/marketplace/google-cloud-build). 
You will set up a trigger that starts the CI/CD workflow when a new tag is applied to the **GitHub** repo managing the  pipeline source code. You will use a fork of this repo as your source GitHub repository.

### Step 1: Create a fork of this repo
[Follow the GitHub documentation](https://help.github.com/en/github/getting-started-with-github/fork-a-repo) to fork [this repo](https://github.com/GoogleCloudPlatform/asl-ml-immersion)

### Step 2: Create a **Cloud Build** trigger

Connect the fork you created in the previous step to your Google Cloud project and create a trigger following the steps in the [Creating GitHub app trigger](https://cloud.google.com/cloud-build/docs/create-github-app-triggers) article. Use the following values on the **Edit trigger** form:

|Field|Value|
|-----|-----|
|Name|[YOUR TRIGGER NAME]|
|Description|[YOUR TRIGGER DESCRIPTION]|
|Event| Tag|
|Source| [YOUR FORK]|
|Tag (regex)|.\*|
|Build Configuration|Cloud Build configuration file (yaml or json)|
|Cloud Build configuration file location| ./notebooks/kubeflow_pipelines/cicd/solutions/cloudbuild_vertex.yaml|


Use the following values for the substitution variables:

|Variable|Value|
|--------|-----|
|_REGION|us-central1|
|_PIPELINE_FOLDER|notebooks/kubeflow_pipelines/cicd/solutions

### Step 3: Trigger the build

To start an automated build [create a new release of the repo in GitHub](https://help.github.com/en/github/administering-a-repository/creating-releases). Alternatively, you can start the build by applying a tag using `git`. 
```
git tag [TAG NAME]
git push origin --tags
```


In [24]:
!git add 

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   cloudbuild_vertex.yaml
	modified:   kfp-cli/Dockerfile
	modified:   kfp_cicd_vertex.ipynb
	modified:   ../solutions/kfp_cicd_vertex.ipynb
	deleted:    ../../pipelines/labs/kfp_pipeline.ipynb
	modified:   ../../pipelines/labs/kfp_pipeline_vertex.ipynb
	modified:   ../../pipelines/labs/kfp_pipeline_vertex_prebuilt.ipynb
	modified:   ../../pipelines/labs/pipeline_vertex/pipeline.py
	modified:   ../../pipelines/labs/pipeline_vertex/training_lightweight_component.py
	modified:   ../../pipelines/labs/pipeline_vertex/tuning_lightweight_component.py
	modified:   ../../pipelines/solutions/kfp_pipeline_vertex_prebuilt.ipynb
	modified:   ../../walkthrough/labs/kfp_walkthrough_vertex.ipynb

Untrack

After running the command above, a build should have been automatically triggered, which you should able to inspect [here](https://console.cloud.google.com/cloud-build/builds).

Copyright 2021 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.